# Index Unique Scan

```sql
SELECT * FROM employees WHERE employee_id = 103;
``` 

# Index Range Scan

- If the data we queried is bounded from one or both sides, the optimizer can use index range scan
- Can be applied to b-tree indexes and bitmap indexes
- Can be applied to unique or non-unique indexes
- Normally, data is stored in ascending order in the indexes
- if the oprimizer finds one or more leading columns with = > < signs, it will use index range scan
- If the query includes an order by or group by clauses with the indexing columns, range scan will not do any sort, it is already sorted. It should not have null value.
- If order by clause has desc keyword, it will read the data in descending order
- you can create your index as dscending
- function based indexes can be accessed as index range scan
- if wildcard characters are written on the right, it will perform index range scan

```sql
-- One side bounded searched
SELECT * FROM SALES WHERE time_id > to_date('01-NOV-01','DD-MON-RR');
 
-- Bounded by both sides
SELECT * FROM SALES WHERE time_id between to_date('01-NOV-00','DD-MON-RR') and to_date('05-NOV-00','DD-MON-RR'); 
 
-- B-Tree index range scan
SELECT * FROM employees where employee_id > 190;
 
-- Index range scan on Non-Unique Index
SELECT * FROM employees where department_id > 80;
 
-- Order by with the indexed column -  sort is processed
SELECT * FROM employees where employee_id > 190 order by email;
 
-- Order by with the indexed column - no sort is processed
SELECT * FROM employees where employee_id > 190 order by employee_id;
 
-- Index range scan descending
SELECT * FROM employees where department_id > 80 order by department_id desc;
 
-- Index range scan with wildcard
SELECT * FROM PRODUCTS WHERE PROD_SUBCATEGORY LIKE 'Accessories%';
SELECT * FROM PRODUCTS WHERE PROD_SUBCATEGORY LIKE '%Accessories';
SELECT * FROM PRODUCTS WHERE PROD_SUBCATEGORY LIKE '%Accessories%';
``` 

# Index Full Scan

- All the rows of the tables are indexes by their indexes
- When the optimizer uses the index full scan?
    - Query has order by clause only with the indexed columns
    - Query has group by clause only with the indexes columns
    - Query requires a sort-merge join

```sql
/* Index usage with order by */
SELECT * FROM departments ORDER BY department_id;
 
/* Index usage with order by, one column of an index - causes index full scan*/
SELECT last_name,first_name FROM employees ORDER BY last_name;
 
/* Index usage with order by, one column of an index - causes unnecessary sort operation*/
SELECT last_name,first_name FROM employees ORDER BY first_name;
 
/* Index usage with order by, but with wrong order - causes unnecessary sort operation */
SELECT last_name,first_name FROM employees ORDER BY first_name,last_name;
 
/* Index usage with order by, with right order of the index - there is no unncessary sort */
SELECT last_name,first_name FROM employees ORDER BY last_name,first_name;
 
/* Index usage with order by, wit unindexed column - there is no unncessary sort */
SELECT last_name,first_name FROM employees ORDER BY last_name,salary;
 
/* Index usage order by - when use * , it performed full table scan */
SELECT * FROM employees ORDER BY last_name,first_name;
 
/* Index usage with group by - using a column with no index leads a full table scan */
SELECT salary,count(*) FROM employees e 
WHERE salary IS NOT NULL
GROUP BY salary;
 
/* Index usage with group by - using indexed columns may lead to a index full scan */
SELECT department_id,count(*) FROM employees e 
WHERE department_id IS NOT NULL
GROUP BY department_id;
 
/* Index usage with group by - using more columns than ONE index has may prevent index full scan */ 
SELECT department_id,manager_id,count(*) FROM employees e 
WHERE department_id IS NOT NULL
GROUP BY department_id, manager_id;
 
/* Index usage with merge join */
SELECT e.employee_id, e.last_name, e.first_name, e.department_id, 
       d.department_name
FROM   employees e, departments d
WHERE  e.department_id = d.department_id;
``` 

# Index Fast Full Scan

- If the query requests only the columns existing in the index, it uses IFF Scan
- Can be applied to both b-tree and bitmap indexes
- Hints can be used to force the optimizer to use IFF scan
- The diff of index full scan vs index fast full scan
    - IFF always reads only from the index | index full scan may read from table
    - IFS reads blocks one by one, sequentially | IFF reads multiple simultaneously, in unordered manner
    - IFF is faster than IFS most of the times
    - IFS can be used to eliminate sorting, IFF cannot

```sql
/* Index Fast Full Scan Usage - Adding a different column 
    than index has will prevent the Index Fast Full Scan */
SELECT e.employee_id, d.department_id, e.first_name,
       d.department_name
FROM   employees e, departments d
WHERE  e.department_id = d.department_id;
 
/* If all the columns are in the index, it may perform
   an Index Fast Full Scan */
SELECT e.employee_id, d.department_id,
       d.department_name
FROM   employees e, departments d
WHERE  e.department_id = d.department_id;
 
/*Index Fast Full Scan can be applied to b-tree indexes, too 
  Even if there is an order by here, it used IFF Scan */
SELECT prod_id from sales order by prod_id;
 
/* Optimizer thinks Index Full Scan is better here*/
SELECT time_id from sales order by time_id;
 
/* Optimizer uses inded Fast Full Scan*/
SELECT time_id from sales;
``` 

# Index Skip Scan

- if you don't use the indexes columns on the where clause, the optimizer will not use the indexes
- we don't create indexes for all the rows because of the costs
- if the sceond, third, column of a composite index is used as an access predicate, the optimizer will consider the index skip scan
- Index skip scan skips the leaves which do not have any chance to have any matching rows
- what are the advantages?
    - helps to reduce the number of indexes
    - decreases the index space
    - Increases the overall performance by reducing the indexes
   

```sql
-- index skip scan : first_name is the second column of an index so it uses ISS
SELECT * FROM employees WHERE first_name = 'Alex';

/*Index skip scan usage with equality operator*/
SELECT * FROM employees WHERE first_name = 'Alex';
 
/* Index range scan occurs if we use the first column of the index */
SELECT * FROM employees WHERE last_name = 'King';
 
/* Using index skip scan with adding a new index */
SELECT * FROM employees WHERE salary BETWEEN 6000 AND 7000;
CREATE INDEX dept_sal_ix ON employees (department_id,salary);
DROP INDEX dept_sal_ix;
 
/* Using index skip scan with adding a new index
   This time the cost increases significantly */
ALTER INDEX customers_yob_bix invisible;
SELECT * FROM customers WHERE cust_year_of_birth BETWEEN 1989 AND 1990;
CREATE INDEX customers_gen_dob_ix ON customers (cust_gender,cust_year_of_birth);
DROP INDEX customers_gen_dob_ix;
ALTER INDEX customers_yob_bix visible;
```

# Index Jon Scan

- If an index stores the columns of a query, the optimizer will perform index fast full scan
- If the combination of multiple indexes store the columns of a query, the optimizer will join them and read the data from that join
- What to know about IJS?
    - The combination of indexes must have every column of the select clause
    - There is no join limit. More than two indexes can be joined together to get the data
    - There might be any index access path before the index join scan
    - If you write ROWID in the select clause, it will not perorm index join scan

```sql
/* Index join scan with two indexes */
SELECT employee_id,email FROM employees;
 
/* Index join scan with two indexes, but with range scan included*/
SELECT last_name,email FROM employees WHERE last_name LIKE 'B%';
 
/* Index join scan is not performed when we add rowid to the select clause */
SELECT rowid,employee_id,email FROM employees;
``` 

# Using Optimizer Hints